In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [ ]:
# Step 1: Data Preprocessing
def preprocess_text(text):
    # Remove URLs, special characters, and convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    return text

In [ ]:
# Load train.csv and test.csv datasets
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train_df.head()

In [ ]:
# Handle missing values in 'keyword' and 'location' columns (if needed)
train_df['keyword'].fillna('', inplace=True)
train_df['location'].fillna('', inplace=True)
test_df['keyword'].fillna('', inplace=True)
test_df['location'].fillna('', inplace=True)

In [ ]:
train_df.head()

In [ ]:
# Preprocess text data
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)

In [ ]:
# Step 2: Feature Engineering (if needed)
# For example, you can create a 'tweet_length' feature
train_df['tweet_length'] = train_df['text'].apply(len)
test_df['tweet_length'] = test_df['text'].apply(len)

In [ ]:
# Step 3: Feature Representation
# Convert text data into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stopwords.words('english'))
X = tfidf_vectorizer.fit_transform(train_df['text']).toarray()
y = train_df['target']

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Model Training and Evaluation
# Train a Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the validation set
y_pred = classifier.predict(X_val)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred))

In [ ]:
# Calculate ROC-AUC score for the validation set
roc_auc = roc_auc_score(y_val, classifier.predict_proba(X_val)[:, 1])
print("ROC-AUC Score:", roc_auc)

In [ ]:
# Step 5: Make Predictions on Test Set
# Convert text data in the test set into numerical vectors using TF-IDF
X_test = tfidf_vectorizer.transform(test_df['text']).toarray()

In [ ]:
# Make predictions on the test set
test_df['target'] = classifier.predict(X_test)

In [ ]:
# Create the prediction output file containing id and target
submission_df = test_df[['id', 'target']]
submission_df.to_csv('submission.csv', index=False)